In [16]:
import numpy as np
from astropy.io import fits
from astropy import stats
import os 
from tqdm import tqdm
from astropy.table import QTable, hstack, Column

In [2]:
# Define names and filenames...
root_dir = '/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/'

In [4]:
def get_files(endswith, root_dir): 

    file_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(endswith):
                print(os.path.join(root, file))
                file_list += [os.path.join(root, file)]

    print('TOTAL NUMBER OF FILES:', len(file_list))
    return(file_list)

files_maps = get_files('_hst_ha.fits', root_dir)
files_errmaps_ = get_files('_hst_ha_err.fits', root_dir)
files_nebs_ = get_files('_nebmask.fits', root_dir)

/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc3627/hst_contsub/ngc3627_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4254/hst_contsub/ngc4254_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1512/hst_contsub/ngc1512_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1433/hst_contsub/ngc1433_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc5068/hst_contsub/ngc5068_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4321/hst_contsub/ngc4321_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1365n/hst_contsub/ngc1365n_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc4303/hst_contsub/ngc4303_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc1087/hst_contsub/ngc1087_hst_ha.fits
/Users/abarnes/Dropbox/work/Smallprojects/galaxies/data_hstha/ngc7496/hst_contsu

In [11]:
files_nebs = []
files_errmaps = []

for file_maps in files_maps: 
    for file_nebs in files_nebs_: 
        if file_maps.split(root_dir)[-1].split('/')[0] == file_nebs.split(root_dir)[-1].split('/')[0]:
            # print(file_maps.split('/')[-1], file_nebs.split('/')[-1])
            if file_maps.split('/')[-1].split('_')[0].lower() != file_nebs.split('/')[-1].split('_')[0].lower():
                print('ERROR: MISMATCHED FILE NAMES')
                print(file_maps.split('/')[-1], file_nebs.split('/')[-1])
            files_nebs += [file_nebs]
            break

    for file_errmaps in files_errmaps_: 
            if file_maps.split(root_dir)[-1].split('/')[0] == file_errmaps.split(root_dir)[-1].split('/')[0]:
                # print(file_maps.split('/')[-1], file_errmaps.split('/')[-1])
                if file_maps.split('/')[-1].split('_')[0].lower() != file_errmaps.split('/')[-1].split('_')[0].lower():
                    print('ERROR: MISMATCHED FILE NAMES')
                    print(file_maps.split('/')[-1], file_nebs.split('/')[-1])
                files_errmaps += [file_errmaps]
                break

ERROR: MISMATCHED FILE NAMES
ngc1365n_hst_ha.fits NGC1365_nebmask.fits
ERROR: MISMATCHED FILE NAMES
ngc2835s_hst_ha.fits NGC2835_nebmask.fits
ERROR: MISMATCHED FILE NAMES
ngc628e_hst_ha.fits NGC0628_nebmask.fits


In [13]:
def get_stats(data): 

    data = data[~np.isnan(data)]
    data = data[data!=0]
    data = data[np.isfinite(data)]

    mean = np.nanmean(data)
    median = np.nanmedian(data)
    p = np.nanpercentile(data, [5, 16, 84, 95])
    std = stats.mad_std(data, ignore_nan=True) 
    return([mean, median, p[0], p[1], p[2], p[3], std])

gals  = []
output_stats = []
output_stats_dict = []

for file_maps, file_errmaps, file_nebs in tqdm(zip(files_maps, files_errmaps, files_nebs)): 

    # Loading files...
    hstha_hdu = fits.open(file_maps)[0]
    muscat_hdu = fits.open(file_nebs)[0]
    hsthaerr_hdu = fits.open(file_errmaps)[0]

    data_map = hstha_hdu.data
    data_err = hsthaerr_hdu.data
    data_mask = muscat_hdu.data

    # # Update arrays
    # muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
    # muscat_hdu.data[muscat_hdu.data==-1] = np.nan

    # # Interpolate masks
    # data_mask_re, _ = reproject_interp(muscat_hdu, hstha_hdu.header)
    # data_mask_re = ~np.isnan(data_mask_re)
    # data_map_masked = hstha_hdu.data[~data_mask_re]

    # Get RMS for whole map... 
    stats_map = get_stats(data_map)
    stats_err = get_stats(data_err)

    gal = file_nebs.split('/')[-1].split('_nebmask.fits')[0]
    gals += [gal]

    output_stats += [[stats_map+stats_err]]
    output_stats_dict += [{'galaxy': gal, 'stats_map': stats_map, 'stats_err': stats_err}]

gals = np.array(gals, dtype='<U12')
output_stats = np.array(output_stats)

gals

17it [00:55,  3.27s/it]


array(['NGC3627', 'NGC4254', 'NGC1512', 'NGC1433', 'NGC5068', 'NGC4321',
       'NGC1365', 'NGC4303', 'NGC1087', 'NGC7496', 'NGC1385', 'NGC2835',
       'IC5332', 'NGC0628', 'NGC1672', 'NGC1300', 'NGC1566'], dtype='<U10')

In [28]:
np.array(output_stats)[:,0,:]

array([[  53.50151443,   30.27719498, -147.20224762,  -77.30212402,
         155.44749451,  288.04224396,  113.87089057,  157.14169312,
         147.61073303,  131.24983215,  137.05072021,  168.15654968,
         198.46812439,   13.11778101],
       [  26.41532516,   13.67222595, -176.02300568, -100.83122284,
         140.16084473,  247.59576263,  119.31268274,  125.007164  ,
         116.1922226 ,   98.04211044,  107.43333435,  132.05342102,
         156.9558754 ,   10.37245239],
       [  12.94887066,    8.33213043, -153.32616119,  -88.17797089,
         108.60825012,  181.51106262,   98.21521426,  140.49621582,
         125.72945404,  112.56422424,  117.53552246,  140.10818237,
         166.89703293,    9.55426233],
       [  21.76677513,    6.36658859, -141.39992218,  -82.15750885,
          99.55355164,  169.79161987,   90.59296003,  232.75340271,
         206.28330994,  181.41798401,  190.56053162,  236.29525635,
         290.65625305,   19.41782242],
       [  28.43364716,   18.

In [32]:
# Get into a astropy table the output of the stats of the maps to .tex

gal_names = gals.copy()
for i, gal in enumerate(gal_names):
    gal = gal.upper()
    if 'NGC' in gal:
        gal_split = gal.split('NGC')
        gal = 'NGC~' + gal_split[1]
    if 'IC' in gal:
        gal_split = gal.split('IC')
        gal = 'IC~' + gal_split[1]
    if '628C' in gal:
        gal_split = gal.split('628C')
        gal = gal_split[0] + '0628C'
    if '628C' in gal:
        gal_split = gal.split('628E')
        gal = gal_split[0] + '0628E'

    gal_names[i] = gal

names=('mean_map', 'median_map', 'p5_map', 'p16_map', 'p84_map', 'p95_map', 'std_map', 
        'mean_noise', 'median_noise', 'p5_noise', 'p16_noise', 'p84_noise', 'p95_noise', 'std_noise')

table1 = Column(gal_names, name='gal_name')
table2 = QTable(np.array(output_stats)[:,0,:], names=names)

table = hstack([table1, table2])
table.sort('gal_name')
table

gal_name,mean_map,median_map,p5_map,p16_map,p84_map,p95_map,std_map,mean_noise,median_noise,p5_noise,p16_noise,p84_noise,p95_noise,std_noise
str10,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC~5332,53.50151443481445,30.27719497680664,-147.2022476196289,-77.3021240234375,155.44749450683594,288.04224395751953,113.87089056998754,157.14169311523438,147.61073303222656,131.2498321533203,137.05072021484375,168.15654968261717,198.46812438964844,13.117781011970113
NGC~0628,26.415325164794922,13.672225952148438,-176.02300567626952,-100.83122283935546,140.16084472656246,247.59576263427726,119.31268274454106,125.00716400146484,116.19222259521484,98.04211044311523,107.43333435058594,132.0534210205078,156.95587539672852,10.37245239301632
NGC~1087,12.948870658874512,8.332130432128906,-153.32616119384767,-88.17797088623047,108.60825012207032,181.5110626220703,98.21521426413737,140.4962158203125,125.72945404052734,112.56422424316406,117.5355224609375,140.10818237304687,166.89703292846679,9.554262332803042
NGC~1300,21.766775131225586,6.366588592529297,-141.39992218017576,-82.15750885009766,99.55355163574217,169.79161987304667,90.59296002937437,232.75340270996094,206.28330993652344,181.41798400878906,190.56053161621094,236.29525634765625,290.65625305175774,19.41782241846571
NGC~1365,28.43364715576172,18.966814041137695,-162.20823669433594,-89.23015594482422,134.80308532714844,226.768310546875,111.10430225506755,117.4937515258789,111.9189224243164,79.40125885009766,102.04927062988281,123.72787506103515,145.9129180908203,9.263956348779404
NGC~1385,24.520835876464844,12.140801429748535,-144.2818603515625,-81.81272430419921,114.69310760498047,204.1169662475586,97.22698622594493,167.12252807617188,154.03271484375,136.3186492919922,142.88092041015625,173.49606323242188,208.6021270751953,12.984352241756092
NGC~1433,40.7706298828125,16.092876434326172,-128.12257385253906,-70.73542785644531,112.61694641113274,203.1985321044922,90.41165028388771,178.81063842773438,158.79603576660156,140.37255783081054,146.68380737304688,184.72971679687498,221.75805816650387,15.303859161058146
NGC~1512,56.40848922729492,29.42946434020996,-164.4867301940918,-87.35549102783203,162.61222717285153,294.2943237304685,122.71733341102113,130.88766479492188,118.13002014160156,104.79788970947266,109.69426727294922,132.99154663085938,157.6205360412597,10.00041619712583
NGC~1566,25.903182983398438,12.778342247009277,-144.48207473754883,-82.3796600341797,117.54818267822263,207.60596466064453,98.98818717371226,167.70968627929688,145.26837158203125,128.82660751342775,134.6124871826172,165.72494506835938,193.43600158691402,13.323534600491055


In [30]:
formats={
        'gal_name': '%s',
        'mean_map': '%.1f',
        'median_map': '%.1f',
        'p5_map': '%.1f',
        'p16_map': '%.1f',
        'p84_map': '%.1f',
        'p95_map': '%.1f',
        'std_map': '%.1f',
        'mean_noise': '%.1f',
        'median_noise': '%.1f',
        'p5_noise': '%.1f',
        'p16_noise': '%.1f',
        'p84_noise': '%.1f',
        'p95_noise': '%.1f',
        'std_noise': '%.1f'
        }

table.write('./tabs/galmap_props.tex', formats=formats, overwrite=True)

In [31]:
# Reorganise to match names in .fits files
gals = table['gal_name']

for i, gal in enumerate(gals):
    if '~' in gal:
        gal_split = gal.split('~')
        gal = gal_split[0] + gal_split[1]
    table['gal_name'][i] = gal

table.write('./tabs/galmap_props.fits', overwrite=True)
table

gal_name,mean_map,median_map,p5_map,p16_map,p84_map,p95_map,std_map,mean_noise,median_noise,p5_noise,p16_noise,p84_noise,p95_noise,std_noise
str10,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
IC5332,15.535592079162598,11.670424461364746,-138.7320297241211,-78.99175201416016,106.20817504882811,173.93586120605465,92.56480672767667,147.56930541992188,133.87310791015625,121.19819641113281,126.02313232421875,147.243408203125,173.427734375,9.140051741376523
NGC0628,10.744853973388672,6.631128311157227,-155.42470703125,-90.61290618896484,108.2556527709961,182.66379394531245,99.21795477369169,140.95657348632812,128.57179260253906,115.91749572753906,120.79995727539062,142.15904235839844,170.80557098388667,8.92772625428875
NGC1087,25.903182983398438,12.778342247009277,-144.48207473754883,-82.3796600341797,117.54818267822263,207.60596466064453,98.98818717371226,167.70968627929688,145.26837158203125,128.82660751342775,134.6124871826172,165.72494506835938,193.43600158691402,13.323534600491055
NGC1300,6.617447376251221,4.163834571838379,-34.082981491088866,-18.708953704833984,28.4156494140625,47.64580154418945,23.401177675375443,62.729129791259766,59.69546127319336,53.28010177612305,55.68147277832031,65.53233337402344,76.75504302978516,4.625932253935362
NGC1365,40.7706298828125,16.092876434326172,-128.12257385253906,-70.73542785644531,112.61694641113274,203.1985321044922,90.41165028388771,178.81063842773438,158.79603576660156,140.37255783081054,146.68380737304688,184.72971679687498,221.75805816650387,15.303859161058146
NGC1385,28.793413162231445,12.20306396484375,-145.44126892089844,-82.9448989868164,117.7815155029297,214.69277954101562,99.07720755533182,176.8992462158203,159.90713500976562,141.75401306152344,148.297607421875,184.10843383789063,218.60796127319335,14.363319805064945
NGC1433,21.766775131225586,6.366588592529297,-141.39992218017576,-82.15750885009766,99.55355163574217,169.79161987304667,90.59296002937437,232.75340270996094,206.28330993652344,181.41798400878906,190.56053161621094,236.29525634765625,290.65625305175774,19.41782241846571
NGC1512,12.948870658874512,8.332130432128906,-153.32616119384767,-88.17797088623047,108.60825012207032,181.5110626220703,98.21521426413737,140.4962158203125,125.72945404052734,112.56422424316406,117.5355224609375,140.10818237304687,166.89703292846679,9.554262332803042
NGC1566,35.89107894897461,11.564538955688477,-149.84842681884766,-85.9038131713867,120.51282958984373,222.57701110839844,101.85157808404523,134.14291381835938,119.60688018798828,106.68421936035156,111.40896606445312,134.8910626220703,160.04492645263667,9.886578697682468
